<a href="https://colab.research.google.com/github/isb-cgc/Community-Notebooks/blob/jacob-dev/MitelmanDB/Mitelman_Fusions_In_TCGA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mitelman Gene Fusions in TCGA

Check out other notebooks at our [Community Notebooks Repository](https://github.com/isb-cgc/Community-Notebooks)!

```
Title: Mitelman Gene Fusions in TCGA  
Author: Jacob Wilson  
Created: 2023-11-01  
URL: https://github.com/isb-cgc/Community-Notebooks/blob/jacob-dev/MitelmanDB/Mitelman_Fusions_In_TCGA.ipynb  
Purpose: We will explore gene fusions in the Mitelman database using BigQuery. With a few basic queries we can select the most common gene fusions specific to any disease type present in the Mitelman database. For demonstration in this notebook, we will be looking at gene fusions associated with Prostate Adenocarcinomas. After creating a list of relevant genes, we will obtain gene expression data from TCGA and build a machine learning model to predict the Primary Gleason Grade.
```

## Initialize Notebook Environment

Before beginning, we first need to load dependencies and authenticate to BigQuery.

## Install Dependencies

In [2]:
# GCP Libraries
from google.cloud import bigquery
from google.colab import auth

from itertools import chain

## Authenticate

In order to utilize BigQuery, we must obtain authorization to BigQuery and Google Cloud.

In [3]:
# if you're using Google Colab, authenticate to gcloud with the following
auth.authenticate_user()

# alternatively, use the gcloud SDK
#!gcloud auth application-default login

## Google project ID

Set your own Google project ID for use with this notebook.

In [4]:
# set the google project that will be billed for this notebook's computations
google_project = 'your_project_id'  ## change this

# set the google project that will be used to store the model and temp table
ML_project = 'your_model_project'  ## change this

# set the dataset for the temporary data table and machine learning model
ML_data = 'your_temp_data_table'  ## change this

## BigQuery Client

In [5]:
# Initialize a client to access the data within BigQuery
if google_project == 'your_project_id':
    print('Please update the project ID with your Google Cloud Project')
else:
    client = bigquery.Client(google_project)

if ML_project == 'your_model_project':
    print('Please update the project ID with your Google Cloud Project')
else:
    model_client = bigquery.Client(model_project)

# set the Mitelman Database project
mitel_proj = 'mitelman-db'
mitel_data = 'prod'

# set the TCGA project
TCGA_proj = 'isb-cgc-bq'
TCGA_data = 'TCGA'

## Exploring Fusions in the Mitelman DB

We will begin by exploring the disease types and gene fusions that are present in the Mitelman database. The database utilizes a coding system where specific disease morphologies and topographics are represented by a unique value stored in the **Koder** table. Using the following queries, we can obtain the codes relevant to our disease of interest.

In [6]:
# query to see all disease types by morphology
query_morph = f'''
SELECT m.Morph, k.Benamning
FROM `{mitel_proj}.{mitel_data}.MolBiolClinAssoc` m
JOIN `{mitel_proj}.{mitel_data}.Koder` k
  ON k.Kod = m.Morph AND k.kodTyp = "MORPH"
GROUP BY m.Morph, k.Benamning
ORDER BY k.Benamning ASC
'''

#print(query_morph)

In [7]:
# query to see all disease types by topography
query_topo = f'''
SELECT m.Topo, k.Benamning
FROM `{mitel_proj}.{mitel_data}.MolBiolClinAssoc` m
JOIN `{mitel_proj}.{mitel_data}.Koder` k
  ON k.Kod = m.Topo AND k.kodTyp = "TOP"
GROUP BY m.Topo, k.Benamning
ORDER BY k.Benamning ASC
'''

#print(query_topo)

In [8]:
# query for gene fusions of all disease types
# gene names that are separated with a double colon "::" represent gene-pair fusions
query_fusions = f'''
SELECT g.Gene, count(g.Gene) AS Count
FROM `{mitel_proj}.{mitel_data}.MolClinGene` g
WHERE g.Gene LIKE "%::%"
GROUP BY g.Gene
ORDER BY Count DESC
'''

#print(query_fusions)

In [9]:
# run the queries and store results in dataframes
morphology_df = client.query(query_morph).result().to_dataframe()
topography_df = client.query(query_topo).result().to_dataframe()
fusions_df = client.query(query_fusions).result().to_dataframe()

In [14]:
print(morphology_df.head())
print(len(morphology_df))

print(topography_df.head())
print(len(topography_df))

print(fusions_df.head())
print(len(fusions_df))

# obtain the morphology and topography codes specific to Prostate Adenocarcinoma
print(morphology_df.loc[morphology_df['Benamning'] == 'Adenocarcinoma'])
print(topography_df.loc[topography_df['Benamning'] == 'Prostate'])

    Morph                                          Benamning
0    3117                              Acinic cell carcinoma
1    1115                          Acute basophilic leukemia
2    1117                        Acute eosinophilic leukemia
3    1112                Acute erythroleukemia (FAB type M6)
4    1602  Acute lymphoblastic leukemia/lymphoblastic lym...
..    ...                                                ...
219  8401               Vascular and perivascular tumor, NOS
220  8499      Vascular and perivascular tumor, special type
221  1710                      Waldenström macroglobulinemia
222  3041                                      Warthin tumor
223  3135                                        Wilms tumor

[224 rows x 2 columns]
    Topo                       Benamning
0   0703                         Adrenal
1   0305                         Bladder
2   0801                           Brain
3   0806                      Brain stem
4   0401                          Breas

These results show that there are a total of 224 morphologies and 50 topographies in the Mitelman database. We will look at fusions involving Adenocarcinomas in the Prostate. The morphology code for Adenocarcinoma is 3111, and the topography code for prostate is 0602. Using these two values, we can construct a list of the top 10 fusions for our target disease.

In [15]:
# query for top ten gene fusions in Prostate Adenocarcinoma
fusions_prostate = f'''
SELECT g.Gene, count(g.Gene) AS Count, m.Morph, k.Benamning, m.Topo
FROM `{mitel_proj}.{mitel_data}.MolClinGene` g
JOIN `{mitel_proj}.{mitel_data}.MolBiolClinAssoc` m
  ON m.RefNo = g.RefNo AND m.InvNo = g.InvNo
JOIN `{mitel_proj}.{mitel_data}.Koder` k
  ON k.Kod = m.Topo AND k.kodTyp = "TOP"
-- we are only considering gene fusions for Prostate Adenocarcinoma
WHERE g.Gene LIKE "%::%" AND m.Morph LIKE "3111" AND m.Topo LIKE "0602"
GROUP BY g.Gene, m.Morph, k.Benamning, m.Topo
ORDER BY Count DESC
LIMIT 10
'''

#print(fusions_prostate)

In [17]:
# run the query and view the top 10 gene fusions
top10_prostate = client.query(fusions_prostate).result().to_dataframe()
print(top10_prostate)

              Gene  Count Morph Benamning  Topo
0     TMPRSS2::ERG     52  3111  Prostate  0602
1    TMPRSS2::ETV4      5  3111  Prostate  0602
2    TMPRSS2::ETV1      5  3111  Prostate  0602
3     SLC45A3::ERG      5  3111  Prostate  0602
4       NDRG1::ERG      4  3111  Prostate  0602
5  OSBPL9::SERINC5      3  3111  Prostate  0602
6    SLC45A3::ELK4      3  3111  Prostate  0602
7       KLK2::ETV1      3  3111  Prostate  0602
8  METTL13::EIF4G3      3  3111  Prostate  0602
9      ADGRL2::AK5      3  3111  Prostate  0602


In [18]:
# convert the list of gene fusion pairs into a string containing individual unique gene names
genes_list = [x.split("::") for x in top10_prostate['Gene']]
genes_set = set(chain.from_iterable(genes_list))
genes_str = ','.join(f"'{gene}'" for gene in genes_set)

print(genes_str)

'ADGRL2','KLK2','METTL13','EIF4G3','AK5','OSBPL9','SERINC5','SLC45A3','ELK4','ERG','ETV4','TMPRSS2','ETV1','NDRG1'


Using the top 10 fusions for Prostate Adenocarcinoma, we have created a list of 14 individual genes (duplicates removed) to explore in TCGA.

## Finding the Genes in TCGA

Now we will explore TCGA to find our list of genes in Prostate Adenocarcinomas. We will first find cases matching our target disease type, then we will create a temporary table containing the gene expression data for these cases.

In [19]:
# query for diseases present in TCGA
query_disease = f'''
SELECT DISTINCT disease_type
FROM `{TCGA_proj}.{TCGA_data}.clinical_gdc_current`
'''

#print(query_disease)

In [20]:
# query for disease sites in TCGA
query_site = f'''
SELECT DISTINCT primary_site
FROM `{TCGA_proj}.{TCGA_data}.clinical_gdc_current`
'''

#print(query_site)

In [21]:
# run the queries and view reults
TCGA_diseases = client.query(query_disease).result().to_dataframe()
print(TCGA_diseases.head())
print(len(TCGA_diseases))

TCGA_site = client.query(query_site).result().to_dataframe()
print(TCGA_site.head())
print(len(TCGA_site))

                                  disease_type
0                    Epithelial Neoplasms, NOS
1                      Squamous Cell Neoplasms
2  Transitional Cell Papillomas and Carcinomas
3                 Adenomas and Adenocarcinomas
4        Cystic, Mucinous and Serous Neoplasms
27
                            primary_site
0                                Bladder
1                                 Breast
2                      Bronchus and lung
3  Other and unspecified parts of tongue
4                                 Larynx
53


There are 27 disease types and 53 primary sites in TCGA. The following query will select the TCGA entries for Prostate Adenocarcinomas.

In [22]:
# query to select all Prostate Adenocarcinomas in TCGA
query_TCGA = f'''
SELECT r.project_short_name,
      r.primary_site,
      r.gene_name,
      p.proj__name,
      p.disease_type,
      p.diag__primary_gleason_grade
FROM `{TCGA_proj}.{TCGA_data}.RNAseq_hg38_gdc_current` r
JOIN `{TCGA_proj}.{TCGA_data}.clinical_gdc_current` p
  ON r.case_gdc_id = p.case_id
--use two critera to find Prostate Adenocarcinomas
WHERE p.disease_type = "Adenomas and Adenocarcinomas"
AND p.primary_site LIKE "%Prostate gland%"
'''

In [23]:
# run the query and view results
TCGA_prostate = client.query(query_TCGA).result().to_dataframe()
print(TCGA_prostate)

         project_short_name    primary_site   gene_name  \
0                 TCGA-PRAD  Prostate gland   SPATA31D1   
1                 TCGA-PRAD  Prostate gland    MIR513A2   
2                 TCGA-PRAD  Prostate gland  AL353651.1   
3                 TCGA-PRAD  Prostate gland   RNU6-555P   
4                 TCGA-PRAD  Prostate gland     MIR548N   
...                     ...             ...         ...   
32879883          TCGA-PRAD  Prostate gland       PELI2   
32879884          TCGA-PRAD  Prostate gland      ZNF503   
32879885          TCGA-PRAD  Prostate gland      B4GAT1   
32879886          TCGA-PRAD  Prostate gland  AC093849.2   
32879887          TCGA-PRAD  Prostate gland       NR2C2   

                       proj__name                  disease_type  \
0         Prostate Adenocarcinoma  Adenomas and Adenocarcinomas   
1         Prostate Adenocarcinoma  Adenomas and Adenocarcinomas   
2         Prostate Adenocarcinoma  Adenomas and Adenocarcinomas   
3         Prostate Aden

## Create a Dataset to Use With the ML Model

A temporary table will be used to store the following data: case identifier, gene name, gene expression values, and Primary Gleason Grade. In the final step of this query, we will pivot the table data creating a column for each gene in our target list.

In [25]:
# query to create a temporary table for use with the ML model
create_tmp_table = f'''
CREATE OR REPLACE TABLE `{ML_project}.{ML_data}.tmp_data` AS
  SELECT * FROM(
    SELECT
      seq.case_barcode,
      seq.gene_name,
      seq.fpkm_uq_unstranded,
      --assign Primary Gleason Grade to an appropriate integer
      CASE clin.diag__primary_gleason_grade
        WHEN "Pattern 1" THEN 1
        WHEN "Pattern 2" THEN 2
        WHEN "Pattern 3" THEN 3
        WHEN "Pattern 4" THEN 4
        WHEN "Pattern 5" THEN 5
      END AS primary_gleason_grade
    FROM `{TCGA_proj}.{TCGA_data}.RNAseq_hg38_gdc_current` seq
    JOIN `{TCGA_proj}.{TCGA_data}.clinical_gdc_current` clin
      ON seq.case_gdc_id = clin.case_id
    WHERE clin.disease_type = "Adenomas and Adenocarcinomas"
    AND clin.primary_site LIKE "%Prostate gland%")
  --transform genes from rows to columns using pivot
  PIVOT(MAX(fpkm_uq_unstranded) for gene_name IN ({genes_str}))
'''
print(create_tmp_table)


CREATE OR REPLACE TABLE `isb-project-zero.jaw_scratch.tmp_data` AS
  SELECT * FROM(
    SELECT
      seq.case_barcode,
      seq.gene_name,
      seq.fpkm_uq_unstranded,
      --assign Primary Gleason Grade to an appropriate integer
      CASE clin.diag__primary_gleason_grade
        WHEN "Pattern 1" THEN 1
        WHEN "Pattern 2" THEN 2
        WHEN "Pattern 3" THEN 3
        WHEN "Pattern 4" THEN 4
        WHEN "Pattern 5" THEN 5
      END AS primary_gleason_grade
    FROM `isb-cgc-bq.TCGA.RNAseq_hg38_gdc_current` seq
    JOIN `isb-cgc-bq.TCGA.clinical_gdc_current` clin
      ON seq.case_gdc_id = clin.case_id
    WHERE clin.disease_type = "Adenomas and Adenocarcinomas"
    AND clin.primary_site LIKE "%Prostate gland%")
  --transform genes from rows to columns using pivot
  PIVOT(MAX(fpkm_uq_unstranded) for gene_name IN ('ADGRL2','KLK2','METTL13','EIF4G3','AK5','OSBPL9','SERINC5','SLC45A3','ELK4','ERG','ETV4','TMPRSS2','ETV1','NDRG1'))



In [26]:
# Run the query. This will create a table in the assigned Google project.
tmp_data = model_client.query(create_tmp_table).result().to_dataframe()

In [27]:
# query to retrieve data from the new table
query_tmp = f'''
  SELECT *
  FROM `{ML_project}.{ML_data}.tmp_data`
  LIMIT 10
'''

# run the query and view results
tmp_table = client.query(query_tmp).result().to_dataframe()
print(tmp_table)

   case_barcode  primary_gleason_grade  ADGRL2       KLK2  METTL13   EIF4G3  \
0  TCGA-CH-5768                      2  5.2915  1072.6219      NaN  11.4251   
1  TCGA-EJ-A8FS                      3  9.3794  1668.2887      NaN   6.1459   
2  TCGA-EJ-5530                      3  2.2175   993.2580      NaN  13.8915   
3  TCGA-G9-7522                      3  3.3433  1326.8446      NaN  10.5922   
4  TCGA-EJ-5521                      3  1.1209   658.5750      NaN   6.9813   
5  TCGA-EJ-A65G                      3  3.6313  2231.2988      NaN   6.6757   
6  TCGA-CH-5750                      3  1.4553   892.1933      NaN  13.1353   
7  TCGA-EJ-7218                      3  5.1444  1788.2251      NaN   7.0529   
8  TCGA-HC-7078                      3  2.5207  1341.2227      NaN   7.7551   
9  TCGA-G9-6342                      3  4.1380  1210.9527      NaN   9.5337   

       AK5   OSBPL9   SERINC5    SLC45A3     ELK4      ERG    ETV4   TMPRSS2  \
0   0.7413  12.3321   50.5366   672.1192  19.8230 

## Create a Machine Learning Model

Using our list of relevant Prostate Adenocarcinoma genes, we will create a random forest classifier to predict the Primary Gleason Grade from the gene expression data of our target genes.

In [28]:
# query to build a random forest classifier model
tree_model_query = f'''
CREATE OR REPLACE MODEL
  `{ML_project}.{ML_data}.treemodel`
OPTIONS
  ( MODEL_TYPE='RANDOM_FOREST_CLASSIFIER',
    NUM_PARALLEL_TREE=HPARAM_RANGE(50,100),
    TREE_METHOD='HIST',
    --split the data randomly into 10% eval, 10% test, and 80% train
    DATA_SPLIT_METHOD='RANDOM',
    DATA_SPLIT_EVAL_FRACTION=0.1,
    DATA_SPLIT_TEST_FRACTION=0.1,
    NUM_TRIALS=3,
    INPUT_LABEL_COLS=['primary_gleason_grade'])
--ignore case identifier and NULL column
AS SELECT * EXCEPT(case_barcode, METTL13)
FROM
  `isb-project-zero.jaw_scratch.tmp_data`
'''

#print(tree_model)

In [29]:
# Run the query. This will store the new model in the Google project.
# NOTE this query may take several minutes to complete.
tree_model = model_client.query(tree_model_query).result()

## Evaluate Model Results

In [30]:
# query to evaluate model performance
eval_query = f'''
SELECT * FROM
  ML.EVALUATE(MODEL `{ML_project}.{ML_data}.treemodel`)
'''

# query for creating a confusion matrix
matrix_query = f'''
SELECT * FROM
  ML.CONFUSION_MATRIX(MODEL `{ML_project}.{ML_data}.treemodel`)
'''

# Run the queries and view results
model_eval = model_client.query(eval_query).result().to_dataframe()
model_matrix = model_client.query(matrix_query).result().to_dataframe()

print(model_eval)
print()
print(model_matrix)

   trial_id  precision    recall  accuracy  f1_score  log_loss   roc_auc
0         1   0.563299  0.570513  0.658537  0.563333  0.857873  0.585271
1         2   0.572917  0.579772  0.682927  0.574206  0.878688  0.579178
2         3   0.563299  0.570513  0.658537  0.563333  0.874047  0.573914

   trial_id expected_label  _2  _3  _4  _5
0         1              3   0   8   5   0
1         1              4   0   9  18   0
2         1              5   0   0   0   1
3         2              3   0   8   5   0
4         2              4   0   8  19   0
5         2              5   0   0   0   1
6         3              3   0   8   5   0
7         3              4   0   9  18   0
8         3              5   0   0   0   1


In [33]:
# query for feature importance in the model
feature_importance_query = f'''
SELECT * FROM
  ML.FEATURE_IMPORTANCE(MODEL `{ML_project}.{ML_data}.treemodel`)
'''

# run the query and view results
model_features = model_client.query(feature_importance_query).result().to_dataframe()
print(model_features)

    trial_id  feature  importance_weight  importance_gain  importance_cover
0          2   ADGRL2               1215         1.343176          8.023765
1          2     KLK2               1211         2.718324         28.955822
2          2   EIF4G3                917         2.185574         14.605371
3          2      AK5                813         3.643522         31.522601
4          2   OSBPL9                759         3.141703         23.775692
5          2  SERINC5                720         2.093802         17.324479
6          2  SLC45A3                847         3.015137         28.898465
7          2     ELK4                493         2.194588         13.797414
8          2      ERG                487         2.270508         15.591376
9          2     ETV4                614         2.762124         25.954397
10         2  TMPRSS2                448         2.195464         20.612444
11         2     ETV1                440         2.103295         21.581250
12         2

## Conclusion

The extensive number of gene fusions in the Mitelman database has allowed us to curate a list of common fusions specific to Prostate Adenocarcinoma. Using these genes, we were easily able to create a random forest classifier in BigQuery that achieved an accuracy of 68% in predicting the Primary Gleason Grade using gene expression data from TCGA. Further experiments can be used to improve this accuracy by incorporating more genes or other features. In this notebook we have demonstrated the usability of the Mitelman and TCGA databases, as well as the ease of creating machine learning models in BigQuery.